In [1]:
%pip install PyTDC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.3 MB/s eta 0:00:00
  Created wheel for PyTDC: filename=PyTDC-0.4.0-py3-none-any.whl size=139519 sha256=29641ac96050f7f1a3ef971451ab9724b1a60dfa3c646839d4559844e2c5e595
  Stored in directory: /root/.cache/pip/wheels/65/da/1a/38e66a2b950c7acc0f41d3bbbc12071653d099baa5402d9c32
Successfully built PyTDC


In [2]:
%pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 15.0 MB/s eta 0:00:00


In [3]:
%pip install pandas-flavor

In [4]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm #progress bar
import rdkit
from rdkit import Chem #Chemistry
from rdkit.Chem import rdMolDescriptors #molecular descriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
rdkit.__version__

import xgboost as xgb
import sklearn
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import fbeta_score, make_scorer
from xgboost.sklearn import XGBRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split #ML training
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error #ML stats
from yellowbrick.regressor import prediction_error, ResidualsPlot
from tdc.single_pred import ADME
from tdc.benchmark_group import admet_group
from tdc import BenchmarkGroup
import warnings
warnings.filterwarnings("ignore")

In [5]:
group = admet_group(path = 'data/')
predictions_list_xgb = []
best_params_list_xgb = []
predictions_list_rf = []
predictions_list_svm = []
best_params_list_svm = []
predictions_list_adb = []
predictions_list_cnn = []

for seed in [1, 2, 3, 4, 5]:
    predictions_xgb = {}
    predictions_rf = {}
    predictions_svm = {}
    predictions_adb = {}
    predictions_cnn = {}

    benchmark = group.get('Lipophilicity_AstraZeneca')
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)

    PandasTools.AddMoleculeColumnToFrame(train, smilesCol='Drug')
    radius=2
    nBits=1024
    ECFP6 = [AllChem.GetMorganFingerprintAsBitVect(x,radius=radius, nBits=nBits) for x in train['ROMol']]
    ecfp6_name = [f'Bit_{i}' for i in range(nBits)]
    ecfp6_bits = [list(l) for l in ECFP6]
    Y = train['Y']
    train = pd.DataFrame(ecfp6_bits, index = train.Drug, columns=ecfp6_name).reset_index(drop = False)
    train['Y'] = Y

    PandasTools.AddMoleculeColumnToFrame(benchmark['test'], smilesCol='Drug')
    radius=2
    nBits=1024
    ECFP6 = [AllChem.GetMorganFingerprintAsBitVect(x,radius=radius, nBits=nBits) for x in benchmark['test']['ROMol']]
    ecfp6_name = [f'Bit_{i}' for i in range(nBits)]
    ecfp6_bits = [list(l) for l in ECFP6]
    Y = benchmark['test']['Y']
    benchmark['test'] = pd.DataFrame(ecfp6_bits, index = benchmark['test'].Drug, columns=ecfp6_name).reset_index(drop = False)
    benchmark['test']['Y'] = Y

    train_X = train.drop(columns = ["Drug","Y"])
    train_y = train.Y
    test_X = benchmark['test'].drop(columns = ["Drug","Y"])
    test_y = benchmark['test'].Y


    #XGBoost + Morgan
    xgb_parameters = {'objective':['reg:squarederror'],
              'booster':['gbtree','gblinear'],
              'learning_rate': [0.1],
              'max_depth': [7,10,15,20],
              'min_child_weight': [10,15,20,25],
              'colsample_bytree': [0.8, 0.9, 1],
              'n_estimators': [300,400,500,600],
              "reg_alpha"   : [0.5,0.2,1],
              "reg_lambda"  : [2,3,5],
              "gamma"       : [1,2,3]}

    xgb_model = XGBRegressor()
    grid_obj_xgb = RandomizedSearchCV(xgb_model, xgb_parameters, cv=5, n_iter=15, scoring='neg_mean_absolute_error', verbose=5, n_jobs=1)
    grid_obj_xgb.fit(train_X, train_y, verbose = 1)
    y_pred_test_xgb = grid_obj_xgb.predict(test_X)
    bp_xgb = grid_obj_xgb.best_params_
    predictions_xgb[name] = y_pred_test_xgb
    predictions_list_xgb.append(predictions_xgb)
    best_params_list_xgb.append(bp_xgb)

    #Random Forest + Morgan
    rf_model = RandomForestRegressor()
    rf_model.fit(train_X, train_y)
    y_pred_test_rf = rf_model.predict(test_X)
    predictions_rf[name] = y_pred_test_rf
    predictions_list_rf.append(predictions_rf)

    #SVM + Morgan
    svm_parameters = {
        'C': [0.1, 1, 10, 100, 1000],
        'epsilon': [0.0001, 0.001, 0.01, 0.1, 0.5],
        'gamma': [0.0001, 0.001, 0.01, 0.1, 1]
    }
    svm_model = SVR(kernel="rbf")
    grid_obj_svm = RandomizedSearchCV(svm_model, svm_parameters, cv=5, n_iter=15, scoring='neg_mean_absolute_error', verbose=5, n_jobs=1)
    grid_obj_svm.fit(train_X, train_y)
    y_pred_test_svm = grid_obj_svm.predict(test_X)
    bp_svm = grid_obj_svm.best_params_
    predictions_svm[name] = y_pred_test_svm
    predictions_list_svm.append(predictions_svm)
    best_params_list_svm.append(bp_svm)

    # AdaBoost + Morgan
    DTR = DecisionTreeRegressor(max_depth=4)
    adb_model = AdaBoostRegressor(n_estimators=300, base_estimator=DTR, learning_rate=1)
    adb_model.fit(train_X, train_y)
    y_pred_test_adb = adb_model.predict(test_X)
    predictions_adb[name] = y_pred_test_adb
    predictions_list_adb.append(predictions_adb)

    #CNN + Morgan
    train_X = train_X.to_numpy()
    test_X = test_X.to_numpy()
    train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], 1)
    test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], 1)
    cnn_model = Sequential()
    cnn_model.add(Conv1D(32, 2, activation="relu", input_shape=(1024,1)))
    cnn_model.add(Flatten())
    cnn_model.add(Dense(64, activation="relu"))
    cnn_model.add(Dense(1))
    cnn_model.compile(loss="mse", optimizer="adam")
    cnn_model.fit(train_X, train_y, batch_size=12, epochs=10, verbose=0)
    y_pred_test_cnn = cnn_model.predict(test_X)
    predictions_cnn[name] = y_pred_test_cnn
    predictions_list_cnn.append(predictions_cnn)

xgb_results = group.evaluate_many(predictions_list_xgb)
rf_results = group.evaluate_many(predictions_list_rf)
svm_results = group.evaluate_many(predictions_list_svm)
adb_results = group.evaluate_many(predictions_list_adb)
cnn_results = group.evaluate_many(predictions_list_cnn)



100%|██████████| 1.47M/1.47M [00:00<00:00, 59.7MiB/s]
Extracting zip file...
Done!
generating training, validation splits...
100%|██████████| 3360/3360 [00:03<00:00, 984.40it/s]


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[03:37:08] WARNING: ../src/learner.cc:767: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight" } are not used.

[CV 1/5] END booster=gblinear, colsample_bytree=0.8, gamma=1, learning_rate=0.1, max_depth=20, min_child_weight=15, n_estimators=500, objective=reg:squarederror, reg_alpha=0.2, reg_lambda=5;, score=-1.010 total time=  13.3s
[03:37:21] WARNING: ../src/learner.cc:767: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight" } are not used.

[CV 2/5] END booster=gblinear, colsample_bytree=0.8, gamma=1, learning_rate=0.1, max_depth=20, min_child_weight=15, n_estimators=500, objective=reg:squarederror, reg_alpha=0.2, reg_lambda=5;, score=-1.082 total time=   8.9s
[03:37:30] WARNING: ../src/learner.cc:767: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight" } are not used.

[CV 3/5] END booster=gblinear, colsample_bytree=0.8, gamma=1, learning_rate=0.1, ma

generating training, validation splits...
100%|██████████| 3360/3360 [00:02<00:00, 1294.79it/s]


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[04:33:15] WARNING: ../src/learner.cc:767: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight" } are not used.

[CV 1/5] END booster=gblinear, colsample_bytree=0.9, gamma=2, learning_rate=0.1, max_depth=15, min_child_weight=20, n_estimators=300, objective=reg:squarederror, reg_alpha=1, reg_lambda=2;, score=-0.931 total time=   4.2s
[04:33:19] WARNING: ../src/learner.cc:767: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight" } are not used.

[CV 2/5] END booster=gblinear, colsample_bytree=0.9, gamma=2, learning_rate=0.1, max_depth=15, min_child_weight=20, n_estimators=300, objective=reg:squarederror, reg_alpha=1, reg_lambda=2;, score=-1.046 total time=   6.3s
[04:33:25] WARNING: ../src/learner.cc:767: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight" } are not used.

[CV 3/5] END booster=gblinear, colsample_bytree=0.9, gamma=2, learning_rate=0.1, max_de

generating training, validation splits...
100%|██████████| 3360/3360 [00:01<00:00, 1732.84it/s]


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[05:26:30] WARNING: ../src/learner.cc:767: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight" } are not used.

[CV 1/5] END booster=gblinear, colsample_bytree=0.8, gamma=3, learning_rate=0.1, max_depth=10, min_child_weight=10, n_estimators=300, objective=reg:squarederror, reg_alpha=1, reg_lambda=2;, score=-0.949 total time=   5.8s
[05:26:36] WARNING: ../src/learner.cc:767: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight" } are not used.

[CV 2/5] END booster=gblinear, colsample_bytree=0.8, gamma=3, learning_rate=0.1, max_depth=10, min_child_weight=10, n_estimators=300, objective=reg:squarederror, reg_alpha=1, reg_lambda=2;, score=-0.970 total time=   4.3s
[05:26:40] WARNING: ../src/learner.cc:767: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight" } are not used.

[CV 3/5] END booster=gblinear, colsample_bytree=0.8, gamma=3, learning_rate=0.1, max_de

generating training, validation splits...
100%|██████████| 3360/3360 [00:01<00:00, 1684.78it/s]


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END booster=gbtree, colsample_bytree=0.8, gamma=3, learning_rate=0.1, max_depth=10, min_child_weight=20, n_estimators=300, objective=reg:squarederror, reg_alpha=1, reg_lambda=3;, score=-0.759 total time=  42.4s
[CV 2/5] END booster=gbtree, colsample_bytree=0.8, gamma=3, learning_rate=0.1, max_depth=10, min_child_weight=20, n_estimators=300, objective=reg:squarederror, reg_alpha=1, reg_lambda=3;, score=-0.678 total time=  42.3s
[CV 3/5] END booster=gbtree, colsample_bytree=0.8, gamma=3, learning_rate=0.1, max_depth=10, min_child_weight=20, n_estimators=300, objective=reg:squarederror, reg_alpha=1, reg_lambda=3;, score=-0.805 total time=  42.7s
[CV 4/5] END booster=gbtree, colsample_bytree=0.8, gamma=3, learning_rate=0.1, max_depth=10, min_child_weight=20, n_estimators=300, objective=reg:squarederror, reg_alpha=1, reg_lambda=3;, score=-0.749 total time=  44.1s
[CV 5/5] END booster=gbtree, colsample_bytree=0.8, gamma=3,

generating training, validation splits...
100%|██████████| 3360/3360 [00:01<00:00, 1747.64it/s]


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END booster=gbtree, colsample_bytree=1, gamma=2, learning_rate=0.1, max_depth=20, min_child_weight=25, n_estimators=600, objective=reg:squarederror, reg_alpha=1, reg_lambda=2;, score=-0.723 total time= 3.4min
[CV 2/5] END booster=gbtree, colsample_bytree=1, gamma=2, learning_rate=0.1, max_depth=20, min_child_weight=25, n_estimators=600, objective=reg:squarederror, reg_alpha=1, reg_lambda=2;, score=-0.760 total time= 3.4min
[CV 3/5] END booster=gbtree, colsample_bytree=1, gamma=2, learning_rate=0.1, max_depth=20, min_child_weight=25, n_estimators=600, objective=reg:squarederror, reg_alpha=1, reg_lambda=2;, score=-0.748 total time= 3.4min
[CV 4/5] END booster=gbtree, colsample_bytree=1, gamma=2, learning_rate=0.1, max_depth=20, min_child_weight=25, n_estimators=600, objective=reg:squarederror, reg_alpha=1, reg_lambda=2;, score=-0.724 total time= 3.5min
[CV 5/5] END booster=gbtree, colsample_bytree=1, gamma=2, learning_

In [6]:
xgb_results

{'lipophilicity_astrazeneca': [0.692, 0.013]}

In [7]:
rf_results

{'lipophilicity_astrazeneca': [0.761, 0.007]}

In [8]:
svm_results

{'lipophilicity_astrazeneca': [0.69, 0.018]}

In [9]:
adb_results

{'lipophilicity_astrazeneca': [0.993, 0.037]}

In [10]:
cnn_results

{'lipophilicity_astrazeneca': [0.764, 0.018]}

In [11]:
predictions_list_xgb

[{'lipophilicity_astrazeneca': array([ 3.1932693 ,  2.9113145 ,  3.374337  ,  2.2548463 ,  3.2992477 ,
          2.5041716 ,  2.3696795 ,  2.07445   ,  1.9137881 ,  0.7485736 ,
          1.6386869 ,  2.7285235 ,  2.3720515 ,  2.4449763 ,  2.6285753 ,
          2.6875887 ,  1.6647558 ,  2.1523025 ,  2.2164915 ,  2.970491  ,
          2.9255726 ,  2.8512168 ,  1.6704937 ,  1.7337857 ,  2.4004884 ,
          3.2568638 ,  0.58819985,  0.61750823,  1.7306359 ,  1.7560626 ,
          0.68523264,  1.8376468 ,  1.7144673 ,  2.635121  ,  2.0484989 ,
          2.8257084 ,  1.8723785 ,  3.6093194 ,  1.9755635 ,  1.6438781 ,
          2.478055  ,  1.8321708 ,  1.0075995 ,  1.1986521 ,  3.0012894 ,
          1.8143848 ,  2.523328  ,  1.9809495 ,  2.3541887 ,  1.9871417 ,
          1.0982043 ,  1.2578062 ,  1.3075868 ,  2.6445966 ,  2.6798153 ,
          3.234515  ,  2.9363785 ,  0.9122224 ,  3.2295616 ,  1.652719  ,
          2.0795937 ,  1.5414419 ,  2.433103  ,  1.9512832 ,  2.0088997 ,
         

In [12]:
predictions_list_rf

[{'lipophilicity_astrazeneca': array([ 2.97396667,  2.8848    ,  2.891     ,  2.158425  ,  3.1758    ,
          2.3479    ,  2.4876    ,  2.01095   ,  1.648     ,  0.5582    ,
          0.76151667,  2.6137    ,  2.3117    ,  2.4487    ,  2.22655833,
          2.49605   ,  1.9068    ,  1.7971    ,  2.40046667,  2.17715   ,
          2.6791    ,  2.58606667,  1.81345   ,  1.8904    ,  2.6491    ,
          3.14720333,  1.2754    ,  1.4552    ,  2.0271    ,  2.0501    ,
          1.2026    ,  1.9235    ,  2.2207    ,  2.63566667,  2.5542    ,
          2.5461    ,  1.593     ,  3.5231    ,  1.7886    ,  1.44659667,
          2.5001    ,  2.166     ,  0.6012    ,  1.0308    ,  2.9858    ,
          1.1246    ,  2.73835   ,  2.32268333,  2.4827    ,  2.025025  ,
          1.18929167,  1.04305   ,  1.23328333,  2.03705   ,  2.251     ,
          2.7248    ,  2.8767    ,  1.36388   ,  2.7511    ,  2.24602   ,
          2.1919    ,  2.5496    ,  2.2751    ,  1.6762    ,  1.8026    ,
         

In [13]:
predictions_list_svm

[{'lipophilicity_astrazeneca': array([ 3.95372591,  3.40869136,  4.41679678,  1.7604119 ,  3.93353833,
          2.63812644,  2.37124533,  1.85506827,  1.71945028,  0.07037576,
          1.47653545,  2.48192487,  1.74518184,  2.15201879,  2.01340954,
          3.18642586,  1.19580998,  1.6811994 ,  3.01849808,  4.0401249 ,
          3.13919006,  3.24933792,  2.64534988,  2.19141002,  2.14393005,
          4.34360546,  0.65272739,  0.30008386,  1.95632092,  1.66446082,
          1.04695827,  3.15210094,  1.76703009,  2.02214539,  2.60069084,
          2.898643  ,  2.17535442,  3.91946125,  1.84029806,  1.60184243,
          3.04475823,  1.76250756,  0.67674406,  2.06432982,  4.35335174,
          2.62206634,  2.89859055,  2.41177102,  2.514407  ,  2.10309033,
          1.29869108,  1.86545064,  2.00127225,  3.50838022,  3.24620817,
          3.5985256 ,  3.15055741,  1.38891412,  4.05645573,  2.26292955,
          1.43788347,  2.13322025,  2.77885851,  1.56873706,  0.81862539,
         

In [14]:
predictions_list_adb

[{'lipophilicity_astrazeneca': array([2.23082009, 2.09416231, 2.165582  , 2.05543774, 2.1386935 ,
         1.45198366, 1.39208516, 2.14416667, 1.92924944, 1.75026026,
         2.00101972, 2.10381579, 2.0286464 , 2.10381579, 2.10381579,
         2.38625597, 1.53629921, 2.13094271, 2.03162856, 2.09343829,
         2.00764801, 2.14429215, 1.99567775, 1.84292431, 1.8968324 ,
         2.09065073, 1.01392886, 1.06975092, 1.27117226, 1.75026026,
         1.05202904, 1.81523417, 1.54624153, 2.06702648, 2.05543774,
         2.13094271, 1.21194753, 2.23144626, 2.30385044, 1.05027143,
         2.04123671, 1.25862069, 1.96352629, 1.26589286, 2.2391278 ,
         1.59602637, 2.15058626, 1.80492608, 2.10009224, 1.83813754,
         1.22038865, 1.40040486, 1.83832381, 1.77028215, 1.83018527,
         2.46249679, 2.14628044, 1.89560597, 1.45198366, 2.05543774,
         1.22038865, 2.0839166 , 2.23082009, 2.04123671, 1.22779956,
         1.77028215, 2.03162856, 1.96352629, 2.05543774, 2.03162856,
     

In [15]:
predictions_list_cnn

[{'lipophilicity_astrazeneca': array([[ 3.4663022 ],
         [ 2.8212326 ],
         [ 4.17431   ],
         [ 2.2585058 ],
         [ 4.4010057 ],
         [ 2.9025311 ],
         [ 2.7686176 ],
         [ 1.9730731 ],
         [ 1.5524524 ],
         [ 0.03796417],
         [ 0.87514675],
         [ 2.7479753 ],
         [ 1.832122  ],
         [ 2.064718  ],
         [ 2.1450188 ],
         [ 2.1294706 ],
         [ 0.9958762 ],
         [ 0.7481611 ],
         [ 2.925446  ],
         [ 3.6602473 ],
         [ 2.3550913 ],
         [ 2.754506  ],
         [ 2.2738702 ],
         [ 1.9831073 ],
         [ 2.039729  ],
         [ 4.4608483 ],
         [ 0.6075537 ],
         [ 0.41214067],
         [ 1.3148453 ],
         [ 1.0683826 ],
         [ 0.31396905],
         [ 2.326838  ],
         [ 1.4977794 ],
         [ 1.2534486 ],
         [ 1.6973131 ],
         [ 3.4098554 ],
         [ 2.0819874 ],
         [ 3.7515614 ],
         [ 1.4628875 ],
         [ 1.5503255 ],
         [ 